In [30]:
import pandas as pd

In [31]:
df = pd.read_csv("train.csv")

In [32]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [33]:
df.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [35]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [36]:
df = df.fillna("")

In [37]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [38]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [39]:
df = df.drop(['id', 'title', 'author'], axis=1)

In [40]:
df.head()

,text,label
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,Ever get the feeling your life circles the rou...,0
2,"Why the Truth Might Get You Fired October 29, ...",1
3,Videos 15 Civilians Killed In Single US Airstr...,1
4,Print \nAn Iranian woman has been sentenced to...,1


In [54]:
import nltk

In [55]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SC\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [52]:
from nltk.corpus import stopwords

In [42]:
from nltk.stem.porter import PorterStemmer

In [43]:
import re

In [44]:
port_stem = PorterStemmer()

In [45]:
port_stem

<PorterStemmer>

In [46]:
port_stem.stem("Hi, THis is Abdul")

'hi, this is abdul'

In [49]:
def stemming(content):
    con = re.sub("[^a-zA-Z]"," ", content)
    con = con.lower()
    con = con.split()
    con = [port_stem.stem(word) for word in con if not word in stopwords.words("english")]
    con = " ".join(con)
    return con

In [56]:
stemming("Hi this is Abdul")

'hi abdul'

In [57]:
df["text"] = df["text"].apply(stemming)

In [63]:
x = df["text"]

In [62]:
y = df["label"]

In [64]:
y.shape

(20800,)

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
x_train , x_test , y_train , y_test = train_test_split(x , y, test_size=0.20)

In [67]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [68]:
vect = TfidfVectorizer()

In [69]:
x_train = vect.fit_transform(x_train)
x_test = vect.transform(x_test)

In [70]:
x_test.shape

(4160, 98211)

In [71]:
from sklearn.tree import DecisionTreeClassifier

In [72]:
model = DecisionTreeClassifier()

In [73]:
model.fit(x_train,y_train)

DecisionTreeClassifier()

In [84]:
prediction = model.predict(x_test)

In [75]:
prediction

array([0, 0, 0, ..., 0, 1, 1], dtype=int64)

In [76]:
model.score(x_test,y_test)

0.8875

In [77]:
import pickle

In [78]:
pickle.dump(vect, open("vector.pkl", "wb"))

In [79]:
pickle.dump(model, open("model.pkl", "wb"))

In [81]:
vector_form = pickle.load(open("vector.pkl", "rb"))

In [82]:
load_model = pickle.load(open("model.pkl", "rb"))

In [83]:
def fake_news(news):
    news = stemming(news)
    input_data = [news]
    vector_form1 = vector_form.transform(input_data)
    prediction = load_model.predict(vector_form1)
    return prediction